In [27]:
import pandas as pd
import numpy as np

In [28]:
titanic = pd.read_csv('train.csv')
t = titanic.set_index(['PassengerId'])
pd.set_option('display.max_rows', None)

In [29]:
def fit(dataset, n_count):
    result = pd.DataFrame(index=t.columns, columns=t.columns)
    for outer_column in t.columns:
        sum = 0
        if outer_column in ['Name', 'Survived']:
            continue
        ff = t[outer_column].value_counts()[t[outer_column].value_counts() > n_count]
        if len(ff) < 1:
            continue
        #ff = t[t[outer_column].duplicated(keep=False)][outer_column].value_counts()
        for i in ff.items():
            percent = round(t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts(normalize=True)*100, 3).max()
            column_value = t[t[outer_column] == i[0]].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
            sum += i[1] * percent
        result[outer_column]['Survived'] = sum / ff.sum()
    
    t_result = result.dropna(axis='columns', how='all').dropna(axis='index', how='all')#.sort_values(by='Survived', ascending=False)
    return t_result

# res = []
# for i in range(5):
#     res.append(fit(i))
# pd.concat(res).sort_values(['Ticket', 'Sex', 'Fare', 'Pclass', 'Embarked', 'Age', 'SibSp', 'Parch'], ascending=False)
#fit(4).transpose().sort_values(by='Survived')

In [35]:
def Search_max(entry, sort_columns, dataset):
    for column in sort_columns.index: # ['Ticket', 'Cabin', 'Sex', 'Fare', 'Pclass', 'Age', 'Embarked', 'SibSp','Parch']
        for value in t[column]: # t["Ticket"]: [1061, A/5 21171 , ....]
            if value == entry[column]: # if 1061 == entry['Ticket']
                percent = round(t[t[column] == value].fillna(-1)['Survived'].value_counts(normalize=True)*100, 2).max()
                column_value = t[t[column] == value].fillna(-1)['Survived'].value_counts().sort_values(ascending=False).head(1).index[0]
                return column, percent, column_value

def Predict(x, sort_columns, dataset):
    entries = []
    for entry in x:
        entries.append({
            'Pclass': entry[1],
            'Name': entry[2],
            'Sex': entry[3],
            'Age': entry[4],
            'SibSp': entry[5],
            'Parch': entry[6],
            'Ticket': entry[7],
            'Fare': entry[8],
            'Cabin': entry[9],
            'Embarked': entry[10]
        })

    response = []
    result = []
    for entry in entries:
        response.append(Search_max(entry, sort_columns, dataset))

    for i in response:
        result.append(i[2])

    return result, response

In [36]:
dataset = pd.read_csv('test.csv')
dataset.head()

dataset2 = pd.read_csv('gender_submission.csv')
dataset2.head(), dataset.head()

dataset_final = dataset.merge(dataset2, on='PassengerId')

x = dataset_final.iloc[:, :-1].values
y = dataset_final.iloc[:, 11].values


In [37]:
n_count = 4
dataset = t
sort_columns = fit(dataset, n_count).transpose().sort_values(by='Survived', ascending=False)
sort_columns

,Survived
Ticket,95.918429
Sex,78.675588
Fare,76.257565
Pclass,67.901302
Embarked,63.779252
Age,63.317413
SibSp,63.30008
Parch,62.866684


In [40]:
from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)
x_test, y_test = x, y

y_pred, meAnalyze = Predict(x_test, sort_columns, dataset)

In [44]:
for i in meAnalyze:
    print(f'Survived is: {i[2]}, Percent is: {i[1]}')
len(meAnalyze)

Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 74.2
Survived is: 0, Percent is: 81.11
Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 100.0
Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 74.2
Survived is: 1, Percent is: 100.0
Survived is: 1, Percent is: 74.2
Survived is: 0, Percent is: 100.0
Survived is: 0, Percent is: 81.11
Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 74.2
Survived is: 0, Percent is: 81.11
Survived is: 0, Percent is: 100.0
Survived is: 0, Percent is: 100.0
Survived is: 0, Percent is: 81.11
Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 74.2
Survived is: 1, Percent is: 74.2
Survived is: 1, Percent is: 100.0
Survived is: 0, Percent is: 81.11
Survived is: 1, Percent is: 74.2
Survived is: 0, Percent is: 100.0
Survived is: 1, Percent is: 100.0
Survived is: 0, Percent is: 100.0
Survived is: 0, Percent is: 100.0
Survived is: 0, Percent is: 81.11
Survived is: 0, Percent is: 81.11
Survived is: 0, Perce

418

In [41]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, y_pred)

print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[227  39]
 [ 36 116]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       266
           1       0.75      0.76      0.76       152

    accuracy                           0.82       418
   macro avg       0.81      0.81      0.81       418
weighted avg       0.82      0.82      0.82       418

Accuracy: 0.8205741626794258
